# interactions

> This module contains functions to analyze interactions between ligands and proteins in PDB.

**Note**
Install [chimerax](https://www.rbvi.ucsf.edu/chimerax/docs/user/index.html) before running

## Add hydrogen using ChimeraX

In [ ]:
import os
import logging
import subprocess


In [ ]:
def protonate_structure(structure: str, out_dir:str) -> tuple[bool, str]:
        """Uses ChimeraX to add hydrogens to the input structure.
        The command being used is `addh hbond true`.

        Args:
            structure: Path to the input structure
            out_dir: Directory where the protonated structure will be saved

        Returns:
            A tuple (success, protonated), where success indicates whether the protonation
            was succcessful or not , and protonated is the path to protonated structure
        """
        logging.info("Protonation started.")

        chimerax_success = False
        protonated_cif = os.path.join(
            out_dir, f'{os.path.basename(structure).split(".")[0]}_h.cif'
        )

        cmd_file = os.path.join(out_dir, "chimera_config.cxc")
        structure = os.path.realpath(structure)
        protonated_cif = os.path.realpath(protonated_cif)

        with open(cmd_file, "w") as f:
            f.write(
                f"""
                open {structure}
                addh hbond true
                save {protonated_cif} format mmcif
                exit
                """
            )
        try:
            subprocess.call(
                ["chimerax", "--nogui", "--cmd", f"open {cmd_file}", "--silent"]
            )
            logging.info("Protonation finished.")
        except Exception:
            return (chimerax_success, protonated_cif)

        if not os.path.isfile(protonated_cif):
            return (chimerax_success, protonated_cif)

        return (chimerax_success, protonated_cif)
